## Treating the Labels as Noisy
### Learning from Noisy Labels using CleanLab

Link: https://github.com/cgnorthcutt/cleanlab/

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
train = pd.read_json('data/interim/train-large.json')
train.head()

,uid,sentiment,text
0,8,neutral,RT @ UAAPconfessions Love looks good on Maddie...
1,12,neutral,Ye Ye ..... ye ??????? We gonna start another ...
2,14,neutral,@ zWffFY9JGklElA1 @ Min _ Of _ Lyching @ thaku...
3,23,negative,~ Caring . ~ Bohot Jyada Caring . ~ Courier wa...
4,24,positive,@ AliHZaidiPTI @ SarfarazA _ 54 What nonesense...


In [3]:
num_classes = len(set(list(train['sentiment']))); num_classes

3

# Refering to : [Twitter-Airlines](https://github.com/martinpella/twitter-airlines/blob/master/shallow_learning.ipynb) for cleaning data. 

In [4]:
# !pip install --upgrade snowballstemmer
# !pip install --upgrade nltk
# !pip install --upgrade scikit-learn

In [5]:
from utils import TextCleaner, CleanTwitter

In [6]:
%time X_train, X_test, y_train, y_test, tfidf_train, tfidf_test = CleanTwitter(train)

CPU times: user 25.2 s, sys: 115 ms, total: 25.3 s
Wall time: 25.4 s


# LearningWithNoisyLabels : [IrisSimple](https://github.com/cgnorthcutt/cleanlab/blob/master/examples/iris_simple_example.ipynb)?

In [7]:
!pip install --upgrade cleanlab

Requirement already up-to-date: cleanlab in /Users/nirant/miniconda3/envs/toys/lib/python3.7/site-packages (0.1.0)


In [8]:
from cleanlab.classification import LearningWithNoisyLabels
from cleanlab.noise_generation import generate_noise_matrix_from_trace
from cleanlab.noise_generation import generate_noisy_labels
from cleanlab.util import value_counts
from cleanlab.latent_algebra import compute_inv_noise_matrix
import cleanlab

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
seed = 37
np.random.seed(seed = seed)

In [10]:
# Not sure if this is how you do it? HOW TO CHANGE THESE VALUES?

# Set the sparsity of the noise matrix.
FRAC_ZERO_NOISE_RATES = 0.5
# A proxy for the fraction of labels that are correct.
avg_trace = 0.67 # ~33% wrong labels. Increasing makes the problem easier.

p_y = value_counts(y_train) # probability of y estimates
noise_matrix = generate_noise_matrix_from_trace(
        K = num_classes,
        trace = num_classes * avg_trace, 
        py = p_y,
        frac_zero_noise_rates = FRAC_ZERO_NOISE_RATES,
    )

# Create noisy labels
s = generate_noisy_labels(y_train, noise_matrix)

In [11]:
noise_matrix

array([[0.57010869, 0.        , 0.        ],
       [0.        , 0.91519804, 0.47530674],
       [0.42989131, 0.08480196, 0.52469326]])

In [12]:
print('WITHOUT confident learning,', end=" ")
m = LogisticRegression(C=4, dual=False, multi_class="auto", solver="lbfgs", max_iter=1000)
_ = m.fit(tfidf_train, y_train)
pred = m.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test), 5))

WITHOUT confident learning, test accuracy: 0.59835


In [13]:
print('WITH confident learning (without noise matrix given),', end=" ")
m_rp = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf = m_rp)
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),5))

WITH confident learning (without noise matrix given), test accuracy: 0.60071


In [14]:
print('WITH confident learning (noise matrix given),', end=" ")
m2_rp = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf = m2_rp)
_ = rp.fit(tfidf_train, y_train, noise_matrix=noise_matrix)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),5))

WITH confident learning (noise matrix given), test accuracy: 0.60376


In [15]:
print('WITH confident learning (noise / inverse noise matrix given),', end=" ")
m3_rp = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf = m3_rp)
_ = rp.fit(tfidf_train, y_train, noise_matrix = noise_matrix, inverse_noise_matrix=compute_inv_noise_matrix(p_y, noise_matrix))
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),5))

WITH confident learning (noise / inverse noise matrix given), test accuracy: 0.60024


In [16]:
print('WITH confident learning (using latent noise matrix estimation),', end=" ")
m = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf = m)
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),5))

WITH confident learning (using latent noise matrix estimation), test accuracy: 0.60212


In [17]:
print('WITH confident learning (using calibrated confident joint),', end=" ")
m = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
rp = LearningWithNoisyLabels(clf=m)
_ = rp.fit(tfidf_train, y_train)
pred = rp.predict(tfidf_test)
print("test accuracy:", round(accuracy_score(pred, y_test),5))

WITH confident learning (using calibrated confident joint), test accuracy: 0.59976


In [18]:
sum(s != y_train)

4029

## Grid Search

In [19]:
# !pip install --upgrade hypopt # this is not a typo, hypopt and hyperopt are different packages

In [20]:
# from hypopt.model_selection import GridSearch
from sklearn.model_selection import ParameterGrid

In [21]:
param_grid = {
    "prune_method": ["prune_by_noise_rate", "prune_by_class", "both"],
    "converge_latent_estimates": [True, False],
}
# Fit LearningWithNoisyLabels across all parameter settings.
params = ParameterGrid(param_grid)
scores = []
for param in params:
    clf = LogisticRegression(solver = 'lbfgs', multi_class = 'auto', max_iter = 1000)
    rp = LearningWithNoisyLabels(clf = clf, **param)
    _ = rp.fit(tfidf_train, s) # s is the noisy y_train labels
    scores.append(accuracy_score(rp.predict(tfidf_test), y_test))

# Print results sorted from best to least
for i in np.argsort(scores)[::-1]:
    print("Param settings:", params[i])
    print(
        "Hinglish dataset test accuracy (using confident learning):\t", 
        round(scores[i], 5),
        "\n"
    )

Param settings: {'prune_method': 'both', 'converge_latent_estimates': False}
Hinglish dataset test accuracy (using confident learning):	 0.47271 

Param settings: {'prune_method': 'both', 'converge_latent_estimates': True}
Hinglish dataset test accuracy (using confident learning):	 0.47176 

Param settings: {'prune_method': 'prune_by_noise_rate', 'converge_latent_estimates': False}
Hinglish dataset test accuracy (using confident learning):	 0.47153 

Param settings: {'prune_method': 'prune_by_class', 'converge_latent_estimates': True}
Hinglish dataset test accuracy (using confident learning):	 0.47012 

Param settings: {'prune_method': 'prune_by_class', 'converge_latent_estimates': False}
Hinglish dataset test accuracy (using confident learning):	 0.46871 

Param settings: {'prune_method': 'prune_by_noise_rate', 'converge_latent_estimates': True}
Hinglish dataset test accuracy (using confident learning):	 0.46776 



# 200 Most Likely Wrong Samples

In [22]:
# !pip install git+https://github.com/cgnorthcutt/rankpruning.git

In [23]:
psx = cleanlab.latent_estimation.estimate_cv_predicted_probabilities(
    tfidf_train,
    s,
    clf=LogisticRegression(max_iter=1000, multi_class="auto", solver="lbfgs"),
)

In [24]:
s = np.asarray(s)
psx = np.asarray(psx)
pd.set_option("display.max_colwidth", 201)
pd.set_option("max_rows", 200)

In [25]:
from cleanlab.pruning import get_noise_indices
import cleanlab

jc, psx = cleanlab.latent_estimation.estimate_confident_joint_and_cv_pred_proba(
    tfidf_train, y_train, rp
)

ordered_label_errors = get_noise_indices(
    s=s, psx=psx, sorted_index_method="normalized_margin"
)

In [26]:
ordered_label_errors

array([ 9732,  9889,  7167, ...,  7053, 10122,  7646])

In [27]:
most_likely_200 = ordered_label_errors[:200]
pd.DataFrame(
    {
        "text": X_train[most_likely_200],
        "label": y_train[most_likely_200],
        "index": most_likely_200,
    }
)

,text,label,index
0,anuview badhai badhai badhai ji haan you are unique thanks good morning happy thursday happy and peacef https t co zrvshfgy,2,9732
1,kylam official yayyyyyy thank you i love you so much,2,9889
2,iamsrk starplus tedtalks i love you so much shahrukh khan you are the best actor and the best father good night https t co jqvbukwkv,2,7167
3,narendramodi congratulations of manniya modi g won ki very very subhkamnaye love you prim minister g,2,643
4,happy birthday to my favorite gemini i love you natali marji,2,2056
5,jenwinget happy birthday my love you are my dream i love you so much tum hamesha khush raho bas yahi dua hai kabh https t co cexkdmnbbo,2,8983
6,happy birthday prachu may god bless you with lots of happiness love you doll muhhhh gprachiprachi,2,117
7,narendramodi humare pradhanmantriji ko bahut bahut shubkamnaye nation good wishes and blessings are always with you and your team,2,57
8,zinda laash hon happy birthday to you many many happy returns of the day may allah bless you,1,4224
9,happy birthday pretty girl thank you for all the good agnes memories cards soon hahahahaha enjoy your day https t co uokhmmws,2,5217
